<a href="https://colab.research.google.com/github/syeongy/Recommendation/blob/main/Perfumes_Item_Based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# 1. Import related library
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [21]:
# Colab
# 2. Read in ratings_df
pf_path = '/content/drive/My Drive/ratings.csv'
ratings_df = pd.read_csv(pf_path)
ratings_df.head()

,userId,perfumeId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [22]:
# Colab
# 3. Read in perfumes_df
pf_path = '/content/drive/My Drive/fianl_perfumes.csv'
perfumes_df = pd.read_csv(pf_path)
perfumes_df.head()

,perfumeId,Name,Brand,Description,Notes,Image URL
0,1,Tihota Eau de Parfum,Indult,"Rapa Nui for sugar, Tihota is, quite simply, ...","Vanilla bean, musks",https://static.luckyscent.com/images/products/...
1,2,Sola Parfum,Di Ser,A tribute to the expanse of space extending f...,"Lavender, Yuzu, Lemongrass, Magnolia, Geraniu...",https://static.luckyscent.com/images/products/...
2,3,Kagiroi Parfum,Di Ser,An aromatic ode to the ancient beauty of Japa...,"Green yuzu, green shikuwasa, sansho seed, cor...",https://static.luckyscent.com/images/products/...
3,4,Velvet Fantasy Eau de Parfum,Montale,Velvet Fantasy is a solar fragrance where cit...,"tangerine, pink pepper, black coffee, leat...",https://static.luckyscent.com/images/products/...
4,5,A Blvd. Called Sunset Eau de Parfum,A Lab on Fire,There's no way A Lab On Fire could relocate t...,"Bergamot, almond, violet, jasmine, leather, s...",https://static.luckyscent.com/images/products/...


In [23]:
ratings_df.columns

Index(['userId', 'perfumeId', 'rating', 'timestamp'], dtype='object')

In [24]:
perfumes_df.columns

Index(['perfumeId', 'Name', 'Brand', 'Description', 'Notes', 'Image URL'], dtype='object')

In [25]:
#4. Merge 2 dataframes together
df_merge = pd.merge(ratings_df, perfumes_df, on='perfumeId', how ='left')
df_merge.head()

,userId,perfumeId,rating,timestamp,Name,Brand,Description,Notes,Image URL
0,1,16,4.0,1217897793,Champs Lunaires Eau de Toilette,Rogue Perfumery,An intangible summer night; Moonlit fields of...,"Tuberose, White Rose Petals, Pomelo, Sandalwo...",https://static.luckyscent.com/images/products/...
1,1,24,1.5,1217895807,Lavande Romaine Eau de Parfum,Perris Monte Carlo,"At harvest time, evenings are filled with it...","Lavender fine, Bleu Cedar, Cassis bourgeons, ...",https://static.luckyscent.com/images/products/...
2,1,32,4.0,1217896246,Escale En Indonesie Eau De Cologne,Les Indemodables,"Perhaps the most classic of all scents, the ""...","Bergamot, lemon, neroli, Indonesian ambergris...",https://static.luckyscent.com/images/products/...
3,1,47,4.0,1217896556,Mango Skin Eau de Parfum,Vilhelm Parfumerie,Have you ever really tasted one? With a singl...,"blackberries, mango, black pepper, wild orris...",https://static.luckyscent.com/images/products/...
4,1,50,4.0,1217896523,Lil Fleur Eau de Parfum,BYREDO,With the swirl of emotions that dominate our ...,"Saffron, cassis, tangerine, leather, Damascen...",https://static.luckyscent.com/images/products/...


In [27]:
df_merge.columns

Index(['userId', 'perfumeId', 'rating', 'timestamp', 'Name', 'Brand',
       'Description', 'Notes', 'Image URL'],
      dtype='object')

In [29]:
# 5. Df statistic
df = df_merge
df.describe()

,userId,perfumeId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105339 entries, 0 to 105338
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userId       105339 non-null  int64  
 1   perfumeId    105339 non-null  int64  
 2   rating       105339 non-null  float64
 3   timestamp    105339 non-null  int64  
 4   Name         48550 non-null   object 
 5   Brand        48550 non-null   object 
 6   Description  48550 non-null   object 
 7   Notes        46797 non-null   object 
 8   Image URL    48550 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 8.0+ MB


In [31]:
# 6. Create user-item interaction matrix
user_item_matrix = df.pivot_table(index='userId', columns='Name', values='rating')
user_item_matrix.head()

Name,00 Auriel Eau de Parfum,01 Nomad Eau de Parfum,02 Owari Eau de Parfum,03 Century Eau de Parfum,03. Apr. 1968 Eau de Parfum,07 Tanoke Eau de Parfum,10 AM Accord Eau de Parfum,10 Corso Como Eau de Parfum,100% Love Eau de Parfum,11 - Semma Eau de Parfum,...,Young Hearts Eau de Parfum,Yoyogi Eau de Toilette,Yu Son Eau de Parfum,Yuzu Eau de Parfum,Zagorsk Eau de Toilette,Zdravetz Eau de Parfum,Zenne Extrait de Parfum,Zeste de Gingembre Eau de Parfum,Zeybek Extrait de Parfum,gentle Fluidity Gold Eau de Parfum
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# 7.1 Creating rating dataframe (mean rating of each perfumes)
ratings = pd.DataFrame(df.groupby('Name')['rating'].mean())
ratings.head()

,rating
Name,
00 Auriel Eau de Parfum,3.200000
01 Nomad Eau de Parfum,4.371429
02 Owari Eau de Parfum,3.928571
03 Century Eau de Parfum,4.166667
03. Apr. 1968 Eau de Parfum,3.000000


In [34]:
# 7.2 New columns for the ratings df
ratings['no_rating'] = df.groupby('Name')['rating'].count() # no_rating = the number of rating
ratings.head()

,rating,no_rating
Name,,
00 Auriel Eau de Parfum,3.200000,15
01 Nomad Eau de Parfum,4.371429,35
02 Owari Eau de Parfum,3.928571,7
03 Century Eau de Parfum,4.166667,42
03. Apr. 1968 Eau de Parfum,3.000000,5


In [36]:
# 7.3 Popular perfumes (base on number of rating)
ratings.sort_values(by='no_rating', ascending=False).head(5)

,rating,no_rating
Name,,
Eau Capitale Eau de Parfum,4.160000,325
Black Caviar Eau de Parfum,4.138264,311
Le Cri De La Lumiere Eau de Parfum,4.454545,308
Maruyama Eau de Parfum,3.659864,294
Noir de Noir Travel Atomizer,4.194828,290


In [38]:
# 8.1 Making recommendation for the 'Eau Capitale Eau de Parfum'
fg_df = user_item_matrix['Eau Capitale Eau de Parfum']

In [39]:
# 8.2 Highest correlation with 'Eau Capitale Eau de Parfum'
sim_fg = user_item_matrix.corrwith(fg_df)

In [40]:
# 8.3 Check the df
sim_fg.head()

Name
00 Auriel Eau de Parfum        0.469801
01 Nomad Eau de Parfum        -0.259706
02 Owari Eau de Parfum         0.000000
03 Century Eau de Parfum      -0.480827
03. Apr. 1968 Eau de Parfum   -0.422577
dtype: float64

In [41]:
# 9. Create a final df
sim_fg = pd.DataFrame(sim_fg, columns=['Correlation'])
final_df = sim_fg.join(ratings['no_rating'])
final_df

,Correlation,no_rating
Name,,
00 Auriel Eau de Parfum,0.469801,15
01 Nomad Eau de Parfum,-0.259706,35
02 Owari Eau de Parfum,0.000000,7
03 Century Eau de Parfum,-0.480827,42
03. Apr. 1968 Eau de Parfum,-0.422577,5
...,...,...
Zdravetz Eau de Parfum,0.559017,7
Zenne Extrait de Parfum,0.013284,156
Zeste de Gingembre Eau de Parfum,NaN,1


In [54]:
# 10. Specify the condition
#final_df.loc[(final_df['no_rating']>100)].sort_values(by='Correlation', ascending=False)
final_df.loc[(final_df['no_rating'] > 30) & (final_df['Correlation'] < 0.9)].sort_values(by='Correlation', ascending=False)

,Correlation,no_rating
Name,,
Au Coeur du Desert Parfum Extrait,0.576556,40
Sublimes Essences Musc Perfume Oil,0.566385,33
Elixir Pour Femme Extrait de Parfum,0.561346,38
Noir de Noir Body Spray Scented Body Spray,0.520750,56
Marinis Parfum,0.479419,41
...,...,...
Beachwood Eau de Parfum,-0.435952,35
03 Century Eau de Parfum,-0.480827,42
Santo Incienso Extrait de Parfum,-0.481830,56
